In [1]:
import glob
import os
import torch

import anndata as ad
import h5py
import scanpy as sc
from scipy.sparse import csr_matrix
from sklearn import model_selection
import yaml

from src.da_utils.data_processing import qc_sc
from src.da_utils import data_loading
from src.da_utils.data_loading import load_spatial, get_selected_dir, get_dset_dir

import pandas as pd
import numpy as np

import seaborn as sns
import pickle

import matplotlib.pyplot as plt


In [2]:
adata_sc = sc.read_h5ad("data/dlpfc/sc_adata/GSE144136.h5ad")

In [12]:
adata_sts = []
for name in glob.glob("data/dlpfc/st_adata/*.h5ad"):
    print(name)
    adata_sts.append(sc.read_h5ad(name))
    

adata_st = ad.concat(adata_sts, join="outer", label="sample")

data/dlpfc/st_adata/spatialLIBD-151508.h5ad
data/dlpfc/st_adata/spatialLIBD-151673.h5ad
data/dlpfc/st_adata/spatialLIBD-151507.h5ad
data/dlpfc/st_adata/spatialLIBD-151672.h5ad
data/dlpfc/st_adata/spatialLIBD-151675.h5ad
data/dlpfc/st_adata/spatialLIBD-151676.h5ad
data/dlpfc/st_adata/spatialLIBD-151674.h5ad
data/dlpfc/st_adata/spatialLIBD-151509.h5ad
data/dlpfc/st_adata/spatialLIBD-151670.h5ad
data/dlpfc/st_adata/spatialLIBD-151671.h5ad
data/dlpfc/st_adata/spatialLIBD-151669.h5ad
data/dlpfc/st_adata/spatialLIBD-151510.h5ad


/projects/wma/AGrEDA/.venv/lib/python3.8/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [16]:
adata_st.obs.groupby("sample").size().mean()

3973.4166666666665

In [15]:
adata_st

AnnData object with n_obs × n_vars = 47681 × 25614
    obs: 'X', 'Y', 'index', 'key', 'subject', 'replicate', 'Cluster', 'sum_umi', 'sum_gene', 'cell_count', 'in_tissue', 'spatialLIBD', 'array_col', 'array_row', 'sample'